In [6]:
import tensorflow as tf

# Transfer learning with tensorflow part 1: feature extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem.

There are two main benefits:

1. Can leverage the existing neural network architecture proven to work on problems similar to our own
2. Can leverage a working neural architectures which has already learned patterns on similar data to our own, then we can adapt those patterns to our own data

## Downloading and becoming one with the data

In [1]:
# Get data(10% of 10 food classes file)
import zipfile 
import os

# Unzip downloaded file
if not "10_food_classes_10_percent.zip" in os.listdir("./"):
    !wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
    zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
    zip_ref.extractall()
    zip_ref.close()

In [2]:
# walk through 10% data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} in {dirpath}")

There are 2 directories and 0 in 10_food_classes_10_percent
There are 10 directories and 0 in 10_food_classes_10_percent/test
There are 0 directories and 250 in 10_food_classes_10_percent/test/fried_rice
There are 0 directories and 250 in 10_food_classes_10_percent/test/ice_cream
There are 0 directories and 250 in 10_food_classes_10_percent/test/ramen
There are 0 directories and 250 in 10_food_classes_10_percent/test/hamburger
There are 0 directories and 250 in 10_food_classes_10_percent/test/sushi
There are 0 directories and 250 in 10_food_classes_10_percent/test/chicken_curry
There are 0 directories and 250 in 10_food_classes_10_percent/test/grilled_salmon
There are 0 directories and 250 in 10_food_classes_10_percent/test/pizza
There are 0 directories and 250 in 10_food_classes_10_percent/test/steak
There are 0 directories and 250 in 10_food_classes_10_percent/test/chicken_wings
There are 10 directories and 0 in 10_food_classes_10_percent/train
There are 0 directories and 75 in 10_fo

## Creating data loaders(preparing the data)
We'll use the `ImageDataGenerator` class to load our images in batches

In [5]:
# Setup data inputs 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE=(224,224)
BATCH_SIZE=32
EPOCHS = 5

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"
train_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)

print("Training Images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

Training Images:
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
